**Assignment 2 - RAG Chatbot**




*   GANESH SINGH SHEKHAWAT - 2023aa05509
*   SAURABH SHARMA - 2023aa05626
*   HEENA SHRIMALI - 2023aa05349
*   AMIT KUMAR SINGH - 2022ac05376



**1. Data Collection & Preprocessing**
*   Download Apple's Financial Statements:




In [4]:
import requests

# URLs of Apple's 2023 and 2024 Annual Reports
urls = {
    '2023': 'https://s2.q4cdn.com/470004039/files/doc_earnings/2023/q4/filing/_10-K-Q4-2023-As-Filed.pdf',
    '2024': 'https://s2.q4cdn.com/470004039/files/doc_earnings/2024/q4/filing/10-Q4-2024-As-Filed.pdf'
}

# Function to download PDFs
def download_pdf(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded {filename}")
    else:
        print(f"Failed to download {filename}")

# Download the reports
for year, url in urls.items():
    download_pdf(url, f'apple_{year}_annual_report.pdf')


Downloaded apple_2023_annual_report.pdf
Downloaded apple_2024_annual_report.pdf


Extract Text from PDFs:

In [5]:
!pip install pymupdf

import fitz  # PyMuPDF

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

# Extract text from each annual report
texts = {}
for year in urls.keys():
    pdf_path = f'apple_{year}_annual_report.pdf'
    texts[year] = extract_text_from_pdf(pdf_path)
    print(f"Extracted text from {pdf_path}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 50.3 MB/s eta 0:00:00
Extracted text from apple_2023_annual_report.pdf
Extracted text from apple_2024_annual_report.pdf


Clean and Structure the Data:

In [6]:
import re

# Function to clean and split text into sections
def clean_and_split_text(text):
    # Remove multiple newlines and excessive whitespace
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\s+', ' ', text)
    # Split text into sections based on headings or other delimiters
    sections = re.split(r'\n\s*\n', text)
    return sections

# Clean and split texts
structured_texts = {}
for year, text in texts.items():
    structured_texts[year] = clean_and_split_text(text)
    print(f"Structured text for {year}")


Structured text for 2023
Structured text for 2024


**Basic RAG Implementation**
*   Convert Financial Documents into Text Chunks:




In [7]:
# Function to chunk text sections into smaller pieces
def chunk_text(sections, chunk_size=500):
    chunks = []
    for section in sections:
        words = section.split()
        for i in range(0, len(words), chunk_size):
            chunk = ' '.join(words[i:i + chunk_size])
            chunks.append(chunk)
    return chunks

# Chunk the structured texts
text_chunks = {}
for year, sections in structured_texts.items():
    text_chunks[year] = chunk_text(sections)
    print(f"Created {len(text_chunks[year])} chunks for {year}")


Created 84 chunks for 2023
Created 129 chunks for 2024


Embed Using a Pre-trained Model:

In [8]:
from sentence_transformers import SentenceTransformer

# Load the pre-trained embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to embed text chunks
def embed_chunks(chunks):
    embeddings = model.encode(chunks, show_progress_bar=True)
    return embeddings

# Embed chunks for each year
embeddings = {}
for year, chunks in text_chunks.items():
    embeddings[year] = embed_chunks(chunks)
    print(f"Generated embeddings for {year}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Generated embeddings for 2023


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Generated embeddings for 2024


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 30.1 MB/s eta 0:00:00


Store and Retrieve Using a Basic Vector Database:

In [22]:
!pip install faiss-cpu
import faiss
import numpy as np

# Function to create a FAISS index
def create_faiss_index(embedding_dim):
    index = faiss.IndexFlatL2(embedding_dim)
    return index

# Create a FAISS index and add embeddings
embedding_dim = embeddings['2023'].shape[1]  # Assuming all embeddings have the same dimension
index = create_faiss_index(embedding_dim)

# Add embeddings to the index
all_embeddings = []
for year in embeddings.keys():
    all_embeddings.extend(embeddings[year])
all_embeddings = np.array(all_embeddings).astype('float32')
index.add(all_embeddings)
print(f"Added {index.ntotal} embeddings to the FAISS index")


Added 213 embeddings to the FAISS index


Implement a Retrieval Function:

In [10]:
# Function to retrieve top-k similar chunks for a query
def retrieve_similar_chunks(query, index, chunks, k=5):
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding).astype('float32'), k)
    results = [chunks[idx] for idx in indices[0]]
    return results

# Combine all chunks for retrieval
all_chunks = []
for year in text_chunks.keys():
    all_chunks.extend(text_chunks[year])

# Example query
query = "What were Apple's net sales in 2024?"
top_k_chunks = retrieve_similar_chunks(query, index, all_chunks)
for i, chunk in enumerate(top_k_chunks, 1):
    print(f"Result {i}:\n{chunk}\n")


Result 1:
non-Pro iPhone models, partially offset by higher net sales of Pro iPhone models. Mac Mac net sales decreased 27% or $10.8 billion during 2023 compared to 2022 due primarily to lower net sales of laptops. iPad iPad net sales decreased 3% or $1.0 billion during 2023 compared to 2022 due primarily to lower net sales of iPad mini and iPad Air, partially offset by the combined net sales of iPad 9th and 10th generation. Wearables, Home and Accessories Wearables, Home and Accessories net sales decreased 3% or $1.4 billion during 2023 compared to 2022 due primarily to lower net sales of Wearables and Accessories. Services Services net sales increased 9% or $7.1 billion during 2023 compared to 2022 due to higher net sales across all lines of business. Apple Inc. | 2023 Form 10-K | 22 Gross Margin Products and Services gross margin and gross margin percentage for 2023, 2022 and 2021 were as follows (dollars in millions): 2023 2022 2021 Gross margin: Products $ 108,803 $ 114,728 $ 105,

In [11]:
!pip install rank_bm25
from rank_bm25 import BM25Okapi

bm25_index = BM25Okapi(all_chunks)  # Using the same 'all_chunks' from before

In [34]:
import random

 #Filters the model's output to avoid misleading or hallucinated responses.
def validate_output(response):
    #  check  hallucinatory patterns
    hallucinated_patterns = ["I think", "Maybe", "It seems", "Could be", "Not sure"]

    #  uncertainty or hallucination  reject
    if any(pattern in response for pattern in hallucinated_patterns):
        return False, "The response seems uncertain or speculative. Please rephrase your query or try again."

    # Check if reponse not right . For Now Simulate
    if random.random() < 0.1:  # Randomly simulate a "bad" answer
        return False, "The response might not be accurate. Please verify it with trusted sources."

    return True, None  # The response is valid



In [35]:
import numpy as np

#Merge chunks by considering content overlap and relevance.
def merge_chunks(embedding_results, bm25_results, k=5):

    merged_results = list(set(embedding_results + bm25_results))

    # Prioritize chunks based on overlap, length, and relevance
    ranked_results = []
    for chunk in merged_results:
        embedding_overlap = sum(1 for e_chunk in embedding_results if e_chunk in chunk)
        bm25_overlap = sum(1 for b_chunk in bm25_results if b_chunk in chunk)
        score = embedding_overlap + bm25_overlap
        ranked_results.append((chunk, score))

    # Sort
    ranked_results.sort(key=lambda x: x[1], reverse=True)
    return [result[0] for result in ranked_results[:k]]


#Dynamically adjust weights on length
def adaptive_weight(query, k=5):
    query_length = len(query.split())
    if query_length < 5:
        #  short queries more BM25
        return 0.3, 0.7
    elif query_length > 20:
        #  long queries favour embeddings
        return 0.7, 0.3
    else:
        # medium-length keep weights balanced
        return 0.5, 0.5


#Retrieve top-k relevant chunks using both embedding-based and BM25-based methods, with adaptive re-ranking."""
def retrieve_similar_chunks_adv(query, index, chunks, bm25_index, k=5):

    # Embedding-based retrieval
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding).astype('float32'), k * 2)
    embedding_results = [chunks[idx] for idx in indices[0]]

    # BM25-based retrieval
    bm25_scores = bm25_index.get_scores(query.split())
    bm25_indices = np.argsort(bm25_scores)[::-1][:k * 2]
    bm25_results = [chunks[idx] for idx in bm25_indices]

    # Merge chunks intelligently
    merged_results = merge_chunks(embedding_results, bm25_results, k)

    # Dynamically adjust the weights for re-ranking
    embedding_weight, bm25_weight = adaptive_weight(query, k)

    # Re-rank results based on adaptive weights
    final_scores = []
    for result in merged_results:
        embedding_score = -distances[0][embedding_results.index(result)] if result in embedding_results else 0  # Negate distance for higher similarity
        bm25_score = bm25_scores[chunks.index(result)] if result in bm25_results else 0
        final_score = embedding_weight * embedding_score + bm25_weight * bm25_score
        final_scores.append(final_score)

    # Sort by final scores and retrieve top-k results
    final_indices = np.argsort(final_scores)[::-1][:k]
    final_results = [merged_results[idx] for idx in final_indices]

    # Display the results
    print(f"Query: {query}")
    for i, result in enumerate(final_results, 1):
            output_valid, output_message = validate_output(final_results)
            if not output_valid:
                return output_message
            print(f"Result {i}:\n{result}\n")

    return final_results



query = "What were Apple's net sales in 2024?"
top_k_chunks = retrieve_similar_chunks_adv(query, index, all_chunks, bm25_index)


Query: What were Apple's net sales in 2024?
Result 1:
U.S. government obligations (as such term is modified below), or both, applicable to the Notes of that series which through the scheduled payment of principal and interest in accordance with their terms will provide money in an amount sufficient to pay the principal or premium, if any, and interest on the Notes on the scheduled due dates therefor. If we effect covenant defeasance with respect to the Notes of any series, the amount in U.S. dollars, or U.S. government obligations (as such term is modified below), or both, on deposit with the trustee will be sufficient, in the opinion of a nationally recognized firm of independent accountants, to pay amounts due on the Notes of that series at the time of the stated maturity but may not be sufficient to pay amounts due on the Notes of that series at the time of the acceleration resulting from such event of default. However, we would remain liable to make payment of such amounts due at t

In [32]:
import re

def validate_query(query):
    # Check  queries
    if not query or query.isspace():
        return False, "Query cannot be empty."

    # profanity check
    profanity_filter =['violence', 'hate', 'explicit', 'offensive']
    if any(word in query.lower() for word in profanity_filter):
        return False, "Query contains inappropriate language."

    # Check for sensitive topics
    sensitive_topics = ["religion", "politics", "sexuality"]
    if any(topic in query.lower() for topic in sensitive_topics):
        return False, "Query touches upon sensitive topics."

    return True, None  # Query is valid

query = "What were Apple's net sales in 2024?"
is_valid, error_message = validate_query(query)

if is_valid:
    #  retrieve results
    top_k_chunks = retrieve_similar_chunks_adv(query, index, all_chunks, bm25_index)
else:
    # Handle invalid query
    print(f"Invalid query: {error_message}")

Query: What were Apple's net sales in 2024?
Result 1:
U.S. government obligations (as such term is modified below), or both, applicable to the Notes of that series which through the scheduled payment of principal and interest in accordance with their terms will provide money in an amount sufficient to pay the principal or premium, if any, and interest on the Notes on the scheduled due dates therefor. If we effect covenant defeasance with respect to the Notes of any series, the amount in U.S. dollars, or U.S. government obligations (as such term is modified below), or both, on deposit with the trustee will be sufficient, in the opinion of a nationally recognized firm of independent accountants, to pay amounts due on the Notes of that series at the time of the stated maturity but may not be sufficient to pay amounts due on the Notes of that series at the time of the acceleration resulting from such event of default. However, we would remain liable to make payment of such amounts due at t